# 📚 Cookbook Based on LangChain Conceptual Documentation 📖


## ❓ What is LangChain?
> LangChain is a framework designed for the development of applications powered by language models.
*[Source](https://blog.langchain.dev/announcing-our-10m-seed-round-led-by-benchmark/#:~:text=LangChain%20is%20a%20framework%20for%20developing%20applications%20powered%20by%20language%20models)*

**Summary**: LangChain simplifies the complexities of working with and building AI models in two key ways:


Most examples in this notebook are inspired by [Greg Kamradt](https://www.youtube.com/watch?v=vGP4pQdCocw) who is a passionately provides contents on LangChain and its usecases. For live examples, visit the [LangChain Project Gallery](https://github.com/gkamradt/langchain-tutorials).

# Use Cases of LangChain


**Main Uses**

1. **Summarization** - Sum up important details from text or chat.
2. **Q&A from Documents** - Answer questions using info from documents.
3. **Data Extraction** - Get structured data from text or user queries.
4. **API Interaction** - Communicate with external systems via APIs.
5. **Tabular Data Queries** - Fetch data from tables or databases.


**Other Roles**
6. **Evaluation** - Check how well your app performs.
7. **Chatbots** - Create conversational bots with memory.
8. **Agents** - Use LLMs for smarter choices.
9. **Understanding Code** - Decode and grasp code.


## Installing Libraries

In [ ]:

%pip install python-dotenv

# installing langchain libraries
%pip install langchain
%pip install openai
%pip install tiktoken
%pip install unstructured

# installing vector stores
%pip install faiss-gpu
%pip install chromadb

## Mounting Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Setting up OpenAI Key

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY', '')

### Make your dispaly a bit wider

In [ ]:
# Run this cell if you want to make your display wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## 1) Summarization

One of the frequently encountered applications of LangChain and Language Models (LLMs) involves text summarization. The utility of this tool extends to summarizing a wide array of content, including calls, articles, books, academic papers, legal documents, user histories, tables, and financial reports. The ability to swiftly condense information is highly valuable

* **Examples** - [Summarizing B2B Sales Calls](https://www.youtube.com/watch?v=DIw4rbpI9ic)
* **Use Cases** - Summarize Articles, Transcripts, Chat History, Slack/Discord, Customer Interactions, Medical Papers, Legal Documents, Podcasts, Tweet Threads, Code Bases, Product Reviews, Financial Documents

### Summaries Of Short Text

For summaries of short texts, the method is straightforward, in fact you don't need to do anything fancy other than simple prompting with instructions


![Alt Text](https://drive.google.com/uc?id=1HUQ4_EPNG7DdrBUaKyUq6pbcMI4yw4gU)


In [ ]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

# Note, the default model is already 'text-davinci-003' but I call it out here explicitly so you know where to change it later if you want
llm = OpenAI(temperature=0, model_name='text-davinci-003', openai_api_key=openai_api_key)

# Create our template
template = """
%INSTRUCTIONS:
Please summarize the following piece of text.
Respond in a manner that a 8 year old would understand.

%TEXT:
{text}
"""

# Create a LangChain prompt template that we can insert values to later
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

Let's let's find a confusing text online from Wikipedia. *[Source](https://en.wikipedia.org/wiki/Network_element#:~:text=According%20to%20the%20Telecommunications%20Act,of%20such%20facility%20or%20equipment.)*

In [ ]:

confusing_text = """
In computer networks, a network element is a manageable logical entity uniting one or more physical devices. This allows distributed devices to be managed in a unified way using one management system.

According to the Telecommunications Act of 1996, the term 'network element' refers to a facility or to equipment used in the provision of a telecommunications service. This term also refers to features, functions, and capabilities that are provided by means of such facility or equipment. This includes items such as subscriber numbers, databases, signaling systems, and information that is sufficient for billing and collection. Alternatively, it's also included if it's used in the transmission, routing, or other provision of a telecommunications service.
"""

Let's take a look at what prompt will be sent to the LLM

In [ ]:
print ("------- Prompt Begin -------")


# create the actual prompt
final_prompt = prompt.format(text=confusing_text)

# how does the final prompt look like
print(final_prompt)

print ("------- Prompt End -------")

------- Prompt Begin -------

%INSTRUCTIONS:
Please summarize the following piece of text.
Respond in a manner that a 8 year old would understand.

%TEXT:

In computer networks, a network element is a manageable logical entity uniting one or more physical devices. This allows distributed devices to be managed in a unified way using one management system.

According to the Telecommunications Act of 1996, the term 'network element' refers to a facility or to equipment used in the provision of a telecommunications service. This term also refers to features, functions, and capabilities that are provided by means of such facility or equipment. This includes items such as subscriber numbers, databases, signaling systems, and information that is sufficient for billing and collection. Alternatively, it's also included if it's used in the transmission, routing, or other provision of a telecommunications service.


------- Prompt End -------


Finally let's pass it through the LLM

In [ ]:
output = llm(final_prompt)
print (output)



Network elements are pieces of equipment or facilities that help computers talk to each other. They help computers send messages and information to each other, like a telephone does. They also help computers keep track of who is sending what messages and how much it costs.


This approach functions effectively; however, when dealing with extended text, it can become cumbersome to handle and may encounter limitations related to the number of tokens.

'text-davinci-003' model has token constraint of 4097 tokens which include prompt and response combined.


Fortunately, LangChain provides built-in support for various summarization methods through their [load_summarize_chain](https://python.langchain.com/en/latest/use_cases/summarization.html) feature.

### Summaries Of Longer Text

*Note: This method will also work for short text too*

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [ ]:
# load data from a url
from langchain.document_loaders import UnstructuredURLLoader

urls = [
   "https://www.reailize.com/post/the-power-of-ticket-automation-accelerating-ticket-resolution-time-unlocking-opex-benefits",
]

loader = UnstructuredURLLoader(urls=urls)

docs = loader.load()
# print(docs[0].page_content[:500])
print(docs)

[Document(page_content='Patryk Debicki\n\nThe Power of Ticket Automation - \u200bAccelerating Ticket Resolution Time & Unlocking OPEX Benefits\n\nIntroduction\n\nEfficient ticket handling becomes paramount as the telecommunications industry embraces new advancements like 5G and technology stack disaggregation. Manual processes and lengthy queueing times can no longer keep up with these cutting-edge networks’ increasing complexity and demands. However, by harnessing the potential of advanced AI/ML algorithms and ticket automation, network operators can experience a transformative shift in their operations. This blog explains the ticket creation and resolution process and delves into the benefits of using ML algorithms and automation at different operational stages to address challenges amplified by the increased complexity of modern networks.\n\nProcess of Ticket Creation and Resolution\n\n1.1 Ticket Creation: Origins and Varieties\n\nTicket origins can vary widely from sources such as 

In [ ]:
## convert to text
text=""

for page in docs:
    text+=page.page_content
text= text.replace('\t', ' ')

print(len(text))


14674


In [ ]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

num_tokens = llm.get_num_tokens(text)

print (f"There are {num_tokens} tokens in your file")

There are 2720 tokens in your file


Then let's check how many tokens are in this document. [get_num_tokens](https://python.langchain.com/en/latest/reference/modules/llms.html#langchain.llms.OpenAI.get_num_tokens) is a nice method for this.

While you could likely stuff this text in your prompt, let's act like it's too big and needs another method.

First we'll need to split it up. This process is called 'chunking' or 'splitting' your text into smaller pieces. I like the [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html) because it's easy to control but there are a [bunch](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html) you can try



In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain


text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=350)
docs = text_splitter.create_documents([text])

print (f"You now have {len(docs)} docs intead of 1 piece of text")

You now have 4 docs intead of 1 piece of text


Next we need to load up a chain which will make successive calls to the LLM for us. Want to see the prompt being used in the chain below? Check out the [LangChain documentation](https://github.com/hwchase17/langchain/blob/master/langchain/chains/summarize/map_reduce_prompt.py)

For information on the difference between chain types, check out this video on [token limit workarounds](https://youtu.be/f9_BWhCI4Zo)

*Note: You could also get fancy and make the first 4 calls of the map_reduce run in parallel too*

In [ ]:
# Get your chain ready to use
chain = load_summarize_chain(llm=llm, chain_type='stuff', verbose=True) # verbose=True optional to see what is getting sent to the LLM

In [ ]:
%time
# Use it. This will run through the 4 documents, summarize the chunks, then get a summary of the summary.
output = chain.run(docs)
print (output)



CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.3 µs


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Patryk Debicki

The Power of Ticket Automation - ​Accelerating Ticket Resolution Time & Unlocking OPEX Benefits

Introduction

Efficient ticket handling becomes paramount as the telecommunications industry embraces new advancements like 5G and technology stack disaggregation. Manual processes and lengthy queueing times can no longer keep up with these cutting-edge networks’ increasing complexity and demands. However, by harnessing the potential of advanced AI/ML algorithms and ticket automation, network operators can experience a transformative shift in their operations. This blog explains the ticket creation and resolution process and delves into the benefits of using ML algorithms and automation at different operational stages to address challenges amplified by the increase

## 2) Question & Answering Using Documents As Context

*[LangChain Question & Answer Docs](https://python.langchain.com/en/latest/use_cases/question_answering.html)*


Extremely common example of chatting with your document.

In order to use LLMs for question and answer we must:

1. Pass the LLM relevant context it needs to answer a question
2. Pass it our question that we want answered

Simplified, this process looks like this "llm(your context + your question) = your answer"

* **Use Cases** - Chat your documents, ask questions to academic papers, create study guides, reference medical information, [PDF.ai](https://pdf.ai/)

### Simple Q&A Example

Here let's review the convention of `llm(your context + your question) = your answer`

In [ ]:
from langchain.llms import OpenAI

openai_api_key = os.getenv('OPENAI_API_KEY', '')
llm = OpenAI(temperature=0, model_name='text-davinci-003', openai_api_key=openai_api_key)


In [ ]:
context = """
Nancy is 20 years old
Shen is 43 years old
Ahmed is 67 years old
"""

question = "Who is under 40 years old?"

Then combine them.

In [ ]:
output = llm(context + question)

# I strip the text to remove the leading and trailing whitespace
print (output.strip())

Nancy is under 40 years old.


### Retrieval Augmented Generation (RAG)

 Using Embeddings, which is going to be a vector representation of different chunks and different documentation. -> Simply 😇 can be explained as your document is going to be converted to a big long vector that is going to be much easier to apply similarity search on!

 In the YouTube tutorial of Greg he often refers to this process as "The VectorStore Dance". It's the process of splitting your text, embedding the chunks, putting the embeddings in a DB, and then querying them.

1. [Load](https://python.langchain.com/docs/modules/data_connection/document_loaders/)
2. [Split/Transform](https://python.langchain.com/docs/modules/data_connection/document_transformers/)
3. [Store/Embed](https://python.langchain.com/docs/modules/data_connection/text_embedding/)
4. [Retrieve](https://python.langchain.com/docs/modules/data_connection/retrievers/)

The goal is to select relevant chunks of our long text, but which chunks do we pull? The most popular method is to pull *similar* texts based off comparing vector embeddings.


![Q & A](https://drive.google.com/uc?id=1iAqI1Rl2QDlsLPtkxfop0fqFC3PoMW98)


If you wanted to do more you would hook this up to a cloud vector database, use a tool like metal and start managing your documents, with external data sources

In [ ]:
%pip install faiss

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [ ]:
from langchain import OpenAI

# Read the information from a website
from langchain.document_loaders import WebBaseLoader

# The vectorstore we'll be using
from langchain.vectorstores import FAISS

# The LangChain component we'll use to get the documents
from langchain.chains import RetrievalQA



# The embedding engine that will convert our text to vectors
from langchain.embeddings.openai import OpenAIEmbeddings

# Load your model
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

The data used is from [Wikipedia](https://en.wikipedia.org/wiki/3GPP)

In [ ]:
# Step 1: Load from wikipedia

loader = WebBaseLoader("https://en.wikipedia.org/wiki/3GPP")
doc = loader.load()

print (f"You have {len(doc)} document")
print (f"You have {len(doc[0].page_content)} characters in that document")

You have 1 document
You have 21437 characters in that document



![RAG](https://drive.google.com/uc?id=1040sgPM0UHB7-Qzeutpq0JVE8JPl0ypM)


In [ ]:

# Step 2: Split documents

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 10)
docs = text_splitter.split_documents(loader.load())

In [ ]:
# Get the total number of characters so we can see the average later
num_total_characters = sum([len(x.page_content) for x in docs])

print (f"Now you have {len(docs)} documents that have an average of {num_total_characters / len(docs):,.0f} characters (smaller pieces)")


Now you have 57 documents that have an average of 375 characters (smaller pieces)


In [ ]:
# Step 3: Embeding

# Get your embeddings engine ready
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Embed your documents and combine with the raw text in a pseudo db. Note: This will make an API call to OpenAI
docsearch = FAISS.from_documents(docs, embeddings)


In [ ]:
# Step 4: Retrive

# Retrival engine
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())


query = "Who are the organizational partners for the 3GPP?"
qa.run(query)

' The seven 3GPP Organizational Partners are the Association of Radio Industries and Businesses (ARIB) from Japan, the Alliance for Telecommunications Industry Solutions (ATIS) from the USA, and the China Communications Standards Association (CCSA) from China.'

## 3) Extraction
*[LangChain Extraction Docs](https://python.langchain.com/en/latest/use_cases/extraction.html)*

(this part is derived from the tutorial by [Greg Kamradt](https://www.youtube.com/watch?v=vGP4pQdCocw))

Extraction is the process of parsing data from a piece of text. This is commonly used with output parsing in order to *structure* our data.


* **Examples** - [OpeningAttributes](https://twitter.com/GregKamradt/status/1646500373837008897)

* **Use Cases:** Extract a structured row from a sentence to insert into a database, extract multiple rows from a long document to insert into a database, extracting parameters from a user query to make an API call

A popular library for extraction is [Kor](https://eyurtsev.github.io/kor/).

![Extraction](https://drive.google.com/uc?id=15cuEtEnc4KUfxaFtW-BttnVa9TLR8sSc)


In [ ]:
# To help construct our Chat Messages
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

# We will be using a chat model, defaults to gpt-3.5-turbo
from langchain.chat_models import ChatOpenAI

# To parse outputs and get structured data back
# StructuredOutputParser, ResponseSchema are being used to get the data out
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

chat_model = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo', openai_api_key=openai_api_key)

### Vanilla Extraction

Let's start off with an easy example. Here I simply supply a prompt with instructions with the type of output I want.

In [ ]:
instructions = """
You will be given a sentence with feeling names, extract those feeling names and assign an emoji to them
Return the feeling  name and emojis in a python dictionary
"""

feeling_names = """
Happiness, Shock, Sadness, Delighfulness, Calm, Chill
"""

In [ ]:
# Make your prompt which combines the instructions w/ the feeling names
prompt = (instructions + feeling_names)

# Call the LLM
output = chat_model([HumanMessage(content=prompt)])

print (output.content)
print (type(output.content))

{
  "Happiness": "😄",
  "Shock": "😱",
  "Sadness": "😢",
  "Delightfulness": "😊",
  "Calm": "😌",
  "Chill": "😎"
}
<class 'str'>


Let's turn this into a proper python dictionary

In [ ]:
output_dict = eval(output.content)

print (output_dict)
print (type(output_dict))

{'Happiness': '😄', 'Shock': '😱', 'Sadness': '😢', 'Delightfulness': '😊', 'Calm': '😌', 'Chill': '😎'}
<class 'dict'>


While this worked this time, it's not a long term reliable method for more advanced use cases and we may need to use **prompt engineering**

### Using LangChain's Response Schema

LangChain's response schema will does two things for us:

1. Autogenerate the a prompt with bonafide format instructions. This is great because I don't need to worry about the prompt engineering side, I'll leave that up to LangChain!

2. Read the output from the LLM and turn it into a proper python object for me

Here I define the schema I want. I'm going to pull out the song and artist that a user wants to play from a pseudo chat message.

In [ ]:
# The schema I want out
response_schemas = [
    ResponseSchema(name="artist", description="The name of the musical artist"),
    ResponseSchema(name="song", description="The name of the song that the artist plays")
]

# The parser that will look for the LLM output in my schema and return it back to me
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
# The format instructions that LangChain makes. Let's look at them
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"artist": string  // The name of the musical artist
	"song": string  // The name of the song that the artist plays
}
```


In [ ]:
# The prompt template that brings it all together
# no need to be worried about the prompt engineering
# Note: This is a different prompt template than before because we are using a Chat Model

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("Given a command from the user, extract the artist and song names \n \
                                                    {format_instructions}\n{user_prompt}")
    ],
    input_variables=["user_prompt"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
feeling_query = prompt.format_prompt(user_prompt="Listening to Hurt from Johnny Cash is interesting")

print (feeling_query.messages[0].content)

Given a command from the user, extract the artist and song names 
                                                     The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"artist": string  // The name of the musical artist
	"song": string  // The name of the song that the artist plays
}
```
Listening to Hurt from Johnny Cash is interesting


In [ ]:
feeling_output = chat_model(feeling_query.to_messages())
output = output_parser.parse(feeling_output.content)

print (output)
print (type(output))

{'artist': 'Johnny Cash', 'song': 'Hurt'}
<class 'dict'>


Awesome, now we have a dictionary that we can use later down the line

<span style="background:#fff5d6">Warning:</span> The parser looks for an output from the LLM in a specific format. Your model may not output the same format every time. Make sure to handle errors with this one. GPT4 and future iterations will be more reliable.

For more advanced parsing check out [Kor](https://eyurtsev.github.io/kor/)

## 4) Interacting with APIs

*[LangChain API Interaction Docs](https://python.langchain.com/en/latest/use_cases/apis.html)*

If the data or action you need is behind an API, you'll need your LLM to interact with APIs


* **Use Cases:** Understand a request from a user and carry out an action, be able to automate more real-world workflows

This topic is closely related to Agents and Plugins, though we'll look at a simple use case for this section. For more information, check out [LangChain + plugins](https://python.langchain.com/en/latest/use_cases/agents/custom_agent_with_plugin_retrieval_using_plugnplai.html) documentation.

### Using DataForSEO
In this example, I am using dataforseo.com
DataForSEO aggregates data from search engines, marketplaces, review platforms, and billions of other websites across the web to provide you with unique insights for building innovative digital marketing solutions.

LangChain's APIChain has the ability to read API documentation and understand which endpoint it needs to call.



In [ ]:
%pip install google-search-results

import os
from langchain.utilities.dataforseo_api_search import DataForSeoAPIWrapper



os.environ["DATAFORSEO_LOGIN"] = "farnoushazour95@gmail.com"
os.environ["DATAFORSEO_PASSWORD"] = "0be13242d73ae956"

wrapper = DataForSeoAPIWrapper()

wrapper.run("Weather in Gliwice?")



'10 Day Weather-Gliwice, Silesian Voivodeship, Poland. As of 9:55 am CEST. Today. 51°/47°. 80%. Thu 19 | Day. 51°. 80%. SE 7 mph.'

In [ ]:
wrapper.run("Weather in Montreal")

'10 Day Weather-Montreal, Quebec, Canada. As of 8:59 pm EDT. Tonight. --/52°. 8%. Thu 19 | Night. 52°. 8%. SSE 6 mph. Cloudy. Low 52F.'

In [ ]:
wrapper.run("What does 1 United States Dollar equals to in Canadian Dollar ")

'1 USD = 1.371165 CAD Oct 19, 2023 01:21 UTC ... Check the currency rates against all the world currencies here. The currency converter below is easy to use and\xa0...'

## 5) Querying Tabular Data

*[LangChain Querying Tabular Data Docs](https://python.langchain.com/en/latest/use_cases/tabular.html)*

The most common type of data in the world sits in tabular form (ok, ok, besides unstructured data). It is super powerful to be able to query this data with LangChain and pass it through to an LLM

* **Examples** - TBD
* **Use Cases:** Use LLMs to query data about users, do data analysis, get real time information from your DBs

One of the best projects
For futher reading check out "Agents + Tabular Data" ([Pandas](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/pandas.html), [SQL](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/sql_database.html), [CSV](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/csv.html))

Let's query an SQLite DB with natural language. We'll look at the [San Francisco Trees](https://data.sfgov.org/City-Infrastructure/Street-Tree-List/tkzw-k3nq) dataset.

![SQL](https://drive.google.com/uc?id=1RFf8e16Z_0LoWINhNmmIk2vFyho0t74N)


In [ ]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain

sqlite_db_path = "/content/drive/MyDrive/ShareYourKnowledge-LangChain/San_Francisco_Trees.db"
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)



In [ ]:
db_chain.run("How many Species of trees are there in San Francisco?")





> Entering new SQLDatabaseChain chain...
How many Species of trees are there in San Francisco?
SQLQuery:SELECT COUNT(DISTINCT "qSpecies") FROM "SFTrees";
SQLResult: [(578,)]
Answer:There are 578 Species of trees in San Francisco.
> Finished chain.


'There are 578 Species of trees in San Francisco.'

This is awesome! There are actually a few steps going on here.

**Steps:**
1. Find which table to use
2. Find which column to use
3. Construct the correct sql query
4. Execute that query
5. Get the result
6. Return a natural language reponse back

### Let's confirm via pandas

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
connection = sqlite3.connect(sqlite_db_path)

# Define your SQL query
query = "SELECT count(distinct qSpecies) FROM SFTrees"

# Read the SQL query into a Pandas DataFrame
df = pd.read_sql_query(query, connection)

# Close the connection
connection.close()

In [ ]:
# Display the result in the first column first cell
print(df.iloc[0,0])

578


## LangChain for Deeper RCA
* [OEM Parsing Package](https://github.com/b-yond-infinite-network/expl-oem-parsing/blob/main/src/expl_oem_parsing/extractor.py)
* Using QAchain to [retrive](https://dbc-f54f5688-178a.cloud.databricks.com/?o=4514799458930737#job/1018187103737867/run/8036234537479) root error.
